In [ ]:
!pip install git+https://github.com/huggingface/transformers.git fastapi uvicorn nest-asyncio pyngrok


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-cts5wnxp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-cts5wnxp
  Resolved https://github.com/huggingface/transformers.git to commit 8f38f58f3de5a35f9b8505e9b48985dce5470985
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10290514 sha256=384a16b2cf65fb4a57b21d8119752e8fc7f72756b17ed921e4c657522b5b027a
  Stored in directory: /tmp/pip-ephem-wheel-cache-yw478gwu/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Succ

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

# Load the processor and model once
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

# Print confirmation
print("Model and Processor Loaded")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 5

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Model and Processor Loaded


In [ ]:
!pip install soundfile

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel
import torch
import scipy.io.wavfile
import tempfile
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Set ngrok authtoken
ngrok.set_auth_token("2pbaUjFQrwYaNaDVqBPr1U1IdE9_6Eq8gfvZCCEgcteYJwZQv")

app = FastAPI()

class MusicInput(BaseModel):
    prompts: list

@app.post("/generate-music/")
async def generate_music(input_data: MusicInput):
    try:
        # Process inputs and generate audio
        inputs = processor(
            text=input_data.prompts,
            padding=True,
            return_tensors="pt",
        )

        audio_values = model.generate(**inputs, max_new_tokens=256)

        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
            sampling_rate = model.config.audio_encoder.sampling_rate
            # Using the exact method you provided
            scipy.io.wavfile.write(
                temp_audio.name,
                rate=sampling_rate,
                data=audio_values[0, 0].numpy()
            )

            return FileResponse(
                temp_audio.name,
                media_type="audio/wav",
                filename="generated_music.wav"
            )

    except Exception as e:
        print(f"Error details: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    try:
        nest_asyncio.apply()
        public_url = ngrok.connect(8000)
        print(f"Public URL: {public_url}")
        uvicorn.run(app, host="0.0.0.0", port=8000)
    except KeyboardInterrupt:
        print("Server is shutting down...")
        ngrok.kill()

INFO:     Started server process [739]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://93da-34-87-69-93.ngrok-free.app" -> "http://localhost:8000"
INFO:     38.224.232.252:0 - "GET / HTTP/1.1" 404 Not Found
